In [8]:
import os
import librosa
import pandas as pd
import numpy as np
from tqdm import tqdm 
# Function to process each WAV file
def process_wav(file_path, sr=16000, frame_duration=7):
    # Load the WAV file
    y, sr = librosa.load(file_path, sr=sr)
    
    # Eliminate the first 15 seconds
    y = y[int(sr*15):]
    
    # Calculate the number of frames
    frame_size = sr * frame_duration
    num_frames = len(y) // frame_size
    
    # Segment the WAV file into frames
    frames = [y[i*frame_size:(i+1)*frame_size] for i in range(num_frames)]
    
    return frames

# Function to get labels for WAV file frames based on participant ID
def get_labels_for_frames(participant_id, data_df, num_frames):
    # Assuming you have loaded the CSV data into a DataFrame named 'data_df'
    # Iterate through the DataFrame to find participant IDs and corresponding labels
    labels = []
    for index, row in data_df.iterrows():
        if str(row['Participant_ID']) == participant_id:
            # print(row['Participant_ID'])
            labels.extend([row['PHQ8_Binary']] * num_frames)
    
    return labels

In [9]:
# Path to the folder containing the WAV files
wav_folder_path = 'E:\\Daicwoz\\after_vad_speech\\'

# Path to the CSV file
csv_file_path = 'combined_data.csv'

# Load the CSV file into a DataFrame
data_df = pd.read_csv(csv_file_path)

# List to store data and labels
data = []
# data = np.array([])  # Initialize an empty numpy array for data
labels = []

# Iterate through each WAV file in the folder
for file_name in tqdm(os.listdir(wav_folder_path)):
    if file_name.endswith('.wav'):
        file_path = os.path.join(wav_folder_path, file_name)
        
        # Extract Participant_ID from the WAV file name
        participant_id = file_name.split('_')[0]
        # print(participant_id)
        
        
        # Process the WAV file
        frames = process_wav(file_path)
        frames = np.array(frames)
        # print(frames.shape)
        
        
        # Concatenate frames to the data array
        # for frame in frames:
        #     data = np.concatenate((data, frame))
        
        # Get labels for WAV file frames based on Participant_ID
        frame_labels = get_labels_for_frames(participant_id, data_df, len(frames))
        
        # Append data and labels
        data.append(frames)
        labels.append(frame_labels)
        
        # print(file_name, len(frames), len(frame_labels), flush=True)
        # print(file_name, len(frames))

# Convert data and labels to numpy arrays
# data = np.array(data)
# labels = np.array(labels)




100%|██████████| 187/187 [00:37<00:00,  5.02it/s]


In [12]:
print(len(data), len(labels))
print(data[0].shape)
data = np.concatenate(data, axis=0)
print(data.shape)
print(labels[0])


187 187
(68, 112000)
(14577, 112000)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [13]:
labels = np.concatenate(labels, axis=0)
print(labels.shape)

(14577,)


In [14]:
np.save('data.npy', data)
np.save('labels.npy', labels)